# Lecture 7 Exercises: Pivot Variables and Special Solutions

This notebook explores null space computation through special solutions and RREF structure.

In [2]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

## Exercise 1 — Build [I_r | F] by Hand

**Goal**: Re-create the RREF block structure manually.

The reduced row echelon form has the structure:
$$
R_{\text{rref}} = [I_r \mid F]
$$

where:
- $I_r$ is the $r \times r$ identity matrix (pivot columns)
- $F$ is the $r \times (n-r)$ free variable coefficient matrix

In [3]:
# Create I_r (2x2 identity)
I_r = np.eye(2)

# Create F (2x3 free variable coefficients)
F = np.array([[2, 3, 4],
              [-1, 4, 5]])

# Construct RREF by horizontal stacking
R = np.hstack([I_r, F])

print("Identity block I_r (2x2):")
print(I_r)
print("\nFree variable block F (2x3):")
print(F)
print("\nRREF matrix R = [I_r | F] (2x5):")
print(R)
print(f"\nShape verification: {R.shape} = (r={I_r.shape[0]}, n={R.shape[1]})")
print(f"Number of free variables: n - r = {R.shape[1]} - {I_r.shape[0]} = {R.shape[1] - I_r.shape[0]}")

Identity block I_r (2x2):
[[1. 0.]
 [0. 1.]]

Free variable block F (2x3):
[[ 2  3  4]
 [-1  4  5]]

RREF matrix R = [I_r | F] (2x5):
[[ 1.  0.  2.  3.  4.]
 [ 0.  1. -1.  4.  5.]]

Shape verification: (2, 5) = (r=2, n=5)
Number of free variables: n - r = 5 - 2 = 3


## Exercise 2 — Construct the Null-Space Matrix

**Goal**: Build the null space basis matrix from RREF.

From the RREF structure $[I_r \mid F]$, the null space matrix is:
$$
N(A) = \begin{bmatrix}
-F \\
I_{n-r}
\end{bmatrix}
$$

This $n \times (n-r)$ matrix contains the special solutions as columns.

In [ ]:
# Get dimensions
r = I_r.shape[0]  # rank
n = R.shape[1]    # total variables
n_minus_r = n - r  # free variables

# Construct null space matrix
N_A = np.vstack([-F, np.eye(n_minus_r)])

print(f"Null space matrix N(A) ({n}x{n_minus_r}):")
print(N_A)
print(f"\nEach column is a special solution (basis vector for null space)")
print(f"\nSpecial solution 1:")
print(N_A[:, 0])
print(f"\nSpecial solution 2:")
print(N_A[:, 1])
print(f"\nSpecial solution 3:")
print(N_A[:, 2])

## Exercise 3 — Verify A @ N(A) = 0

**Goal**: Verify that our null space matrix satisfies $A \cdot N(A) = 0$.

We'll construct a full matrix $A$ from our RREF and verify the null space property.

In [ ]:
# For this exercise, we'll use R as our matrix A
# (In practice, A would reduce to R via row operations)
A = R

# Compute A @ N(A)
result = A @ N_A

print("Matrix A (using RREF):")
print(A)
print(f"\nNull space matrix N(A):")
print(N_A)
print(f"\nA @ N(A) should be zero:")
print(result)
print(f"\nVerification (all entries ≈ 0): {np.allclose(result, 0)}")
print(f"Maximum absolute error: {np.max(np.abs(result)):.2e}")

## Exercise 4 — Compute Rank & Null Space with SciPy

**Goal**: Use SciPy's built-in functions to compute rank and null space, then compare with our manual construction.

In [ ]:
# Create a test matrix
A_test = np.array([[1, 2, 3, 4],
                   [2, 4, 6, 8],
                   [0, 0, 2, 1],
                   [0, 0, 0, 0]])

# Compute rank using matrix_rank
rank = np.linalg.matrix_rank(A_test)
print(f"Matrix A:")
print(A_test)
print(f"\nRank: {rank}")
print(f"Dimensions: {A_test.shape[0]}x{A_test.shape[1]}")
print(f"Number of free variables: {A_test.shape[1] - rank}")

# Compute null space using SVD
U, s, Vh = np.linalg.svd(A_test)

# Null space vectors are the right singular vectors corresponding to zero singular values
# Find indices where singular values are (numerically) zero
tol = 1e-10
null_mask = s < tol
num_zero = np.sum(null_mask)

print(f"\nSingular values:")
print(s)
print(f"\nNumber of (near-)zero singular values: {A_test.shape[1] - len(s) + num_zero}")

# Extract null space from Vh (last n-r rows)
n_vars = A_test.shape[1]
null_space_scipy = Vh[rank:, :].T  # Transpose to get column vectors

print(f"\nNull space basis from SVD ({null_space_scipy.shape[0]}x{null_space_scipy.shape[1]}):")
print(null_space_scipy)

# Verify
print(f"\nVerification A @ N(A) ≈ 0:")
result_scipy = A_test @ null_space_scipy
print(result_scipy)
print(f"All entries near zero: {np.allclose(result_scipy, 0)}")

## Exercise 5 — Compare Manual vs SciPy Null Space

**Goal**: Understand that null space bases can look different but span the same space.

In [ ]:
# Use our earlier example with known RREF structure
A_example = R  # This is already in RREF
N_manual = N_A  # Our manually constructed null space

# Compute null space with SciPy
U, s, Vh = np.linalg.svd(A_example)
rank_example = np.sum(s > 1e-10)
N_scipy = Vh[rank_example:, :].T

print("Manual null space basis:")
print(N_manual)
print(f"\nSciPy null space basis:")
print(N_scipy)

# Both should satisfy A @ N = 0
print(f"\nManual: A @ N_manual ≈ 0: {np.allclose(A_example @ N_manual, 0)}")
print(f"SciPy:  A @ N_scipy ≈ 0:  {np.allclose(A_example @ N_scipy, 0)}")

# Check if they span the same space by checking if each basis can express the other
# If N_scipy columns are in span of N_manual, then N_scipy = N_manual @ C for some C
try:
    C = np.linalg.lstsq(N_manual, N_scipy, rcond=None)[0]
    reconstruction = N_manual @ C
    same_span = np.allclose(reconstruction, N_scipy)
    print(f"\nBases span the same space: {same_span}")
except:
    print("\nCould not verify if bases span the same space (possibly different dimensions)")

## Exercise 6 — Random Matrix Experiment

**Goal**: Explore the relationship between rank and null space dimension for random matrices.

In [ ]:
# Generate random matrices and analyze their null spaces
m, n = 5, 8  # m rows, n columns
num_experiments = 100

ranks = []
null_dims = []

for _ in range(num_experiments):
    # Generate random matrix
    A_random = np.random.randn(m, n)
    
    # Compute rank
    r = np.linalg.matrix_rank(A_random)
    
    # Null space dimension
    null_dim = n - r
    
    ranks.append(r)
    null_dims.append(null_dim)

ranks = np.array(ranks)
null_dims = np.array(null_dims)

print(f"Random matrix experiments (m={m}, n={n}):")
print(f"\nRank statistics:")
print(f"  Min: {ranks.min()}, Max: {ranks.max()}, Mean: {ranks.mean():.2f}")
print(f"\nNull space dimension statistics:")
print(f"  Min: {null_dims.min()}, Max: {null_dims.max()}, Mean: {null_dims.mean():.2f}")
print(f"\nRank-nullity theorem verification (n = r + null_dim):")
print(f"  n = {n}")
print(f"  r + null_dim always equals n: {np.all(ranks + null_dims == n)}")

# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(ranks, bins=range(min(ranks), max(ranks)+2), edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rank')
axes[0].set_ylabel('Frequency')
axes[0].set_title(f'Rank Distribution (m={m}, n={n})')
axes[0].axvline(ranks.mean(), color='red', linestyle='--', label=f'Mean = {ranks.mean():.2f}')
axes[0].legend()

axes[1].hist(null_dims, bins=range(min(null_dims), max(null_dims)+2), edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Null Space Dimension')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Null Space Dimension Distribution')
axes[1].axvline(null_dims.mean(), color='red', linestyle='--', label=f'Mean = {null_dims.mean():.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nObservation: Random matrices typically have full rank (rank = min(m,n) = {min(m,n)})")

## Exercise 7 — Rank-Deficient Matrix Construction

**Goal**: Create a matrix with specified rank and verify its null space dimension.

In [ ]:
def create_rank_deficient_matrix(m, n, desired_rank):
    """
    Create an m×n matrix with exactly the desired rank.
    
    Strategy: A = U @ S @ Vh where S has exactly 'desired_rank' non-zero values
    """
    if desired_rank > min(m, n):
        raise ValueError(f"Rank cannot exceed min(m,n) = {min(m,n)}")
    
    # Create random orthogonal matrices
    U, _ = np.linalg.qr(np.random.randn(m, m))
    V, _ = np.linalg.qr(np.random.randn(n, n))
    
    # Create diagonal matrix with desired_rank non-zero values
    S = np.zeros((m, n))
    for i in range(desired_rank):
        S[i, i] = np.random.uniform(1, 5)  # Random positive values
    
    # Construct matrix
    A = U @ S @ V.T
    
    return A

# Create matrices with different ranks
m, n = 4, 6
for desired_rank in [1, 2, 3, 4]:
    A = create_rank_deficient_matrix(m, n, desired_rank)
    actual_rank = np.linalg.matrix_rank(A)
    null_dim = n - actual_rank
    
    print(f"\nDesired rank: {desired_rank}")
    print(f"Actual rank:  {actual_rank}")
    print(f"Null space dimension: {null_dim}")
    print(f"Rank-nullity check: {actual_rank} + {null_dim} = {actual_rank + null_dim} (should be {n})")
    
    # Compute and verify null space
    U, s, Vh = np.linalg.svd(A)
    N = Vh[actual_rank:, :].T
    
    if N.shape[1] > 0:
        verification = np.allclose(A @ N, 0)
        print(f"Null space verification A @ N ≈ 0: {verification}")

## Summary

Key takeaways:

1. **RREF Structure**: The reduced row echelon form $[I_r \mid F]$ directly reveals the null space

2. **Null Space Formula**: $N(A) = \begin{bmatrix} -F \\ I_{n-r} \end{bmatrix}$ gives a basis for the null space

3. **Rank-Nullity Theorem**: $\text{rank}(A) + \dim(N(A)) = n$ (always holds)

4. **Special Solutions**: Each column of $N(A)$ is a special solution obtained by setting one free variable to 1

5. **Computational Methods**: SVD provides a numerically stable way to compute null spaces

6. **Non-Uniqueness**: Null space bases are not unique, but they all span the same subspace